## SENTIMENT ANALYSIS PILPRES 2024 PADA MEDIA SOSIAL TWITTER DENGAN NAIVE BAYES CLASSIFIER

**MUCHAMMAD FAHD ISHAMUUDIN**


**41155050180048**

In [1]:
%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import collections
import nltk
from nltk.tokenize import word_tokenize
from nltk.sentiment import SentimentIntensityAnalyzer


In [2]:
pilpres = pd.read_csv('2022pilpres.csv')
print(pilpres.head())
print(pilpres.tail())
print(pilpres.columns)

  Unnamed: 0                       date            id  \
0          0  2022-12-30 23:50:32+00:00  1.608974e+18   
1          1  2022-12-30 23:50:16+00:00  1.608974e+18   
2          2  2022-12-30 23:40:27+00:00  1.608971e+18   
3          3  2022-12-30 23:36:07+00:00  1.608970e+18   
4          4  2022-12-30 23:26:47+00:00  1.608968e+18   

                                               tweet         username  
0  Berkat kinerja di BUMN, serta kedekatan dengan...        Fuad_Lha9  
1  @ganjarpranowo menggandeng ulama hingga tokoh ...      iwan_tata98  
2  Mantap 👍👍\nElektabilitas Menteri BUMN Erick Th...        Lima8Satu  
3  Baru Lolos Sebagai Peserta Pemilu 2024, Partai...  21BeritaTerkini  
4  Agenda politik negara tuan rumah Piala Dunia 2...      rezakaisar3  
      Unnamed: 0                       date            id  \
93877      93874  2022-01-01 03:05:00+00:00  1.477114e+18   
93878      93875  2022-01-01 02:24:46+00:00  1.477103e+18   
93879      93876  2022-01-01 02:08:17+00:0

In [3]:
len(pilpres)

93882

In [13]:
pilpres.loc[45,'tweet']

'Megawati Soekarnoputri, diyakini bakal tetap mencalonkan Ketua DPR RI Puan Maharani sebagai capres di Pilpres 2024. Puan akan melaju menjadi capres &amp; berhadapan DGN Anies Baswedan.\n\nPuan Capres PDIP\nhttps://t.co/g367x0BM1P'

In [14]:
x = 'Megawati Soekarnoputri, diyakini bakal tetap mencalonkan Ketua DPR RI Puan Maharani sebagai capres di Pilpres 2024. Puan akan melaju menjadi capres &amp; berhadapan DGN Anies Baswedan.\n\nPuan Capres PDIP\nhttps://t.co/g367x0BM1P'
print(x.lower())

megawati soekarnoputri, diyakini bakal tetap mencalonkan ketua dpr ri puan maharani sebagai capres di pilpres 2024. puan akan melaju menjadi capres &amp; berhadapan dgn anies baswedan.

puan capres pdip
https://t.co/g367x0bm1p


In [20]:
print(pilpres.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 92308 entries, 0 to 92307
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  92308 non-null  object 
 1   date        92308 non-null  object 
 2   id          92305 non-null  float64
 3   tweet       92305 non-null  object 
 4   username    92302 non-null  object 
dtypes: float64(1), object(4)
memory usage: 3.5+ MB
None


In [21]:
del pilpres['Unnamed: 0']


In [39]:
print(pilpres.info())
len(pilpres)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 92308 entries, 0 to 92307
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   date      92308 non-null  object 
 1   id        92305 non-null  float64
 2   tweet     92305 non-null  object 
 3   username  92302 non-null  object 
dtypes: float64(1), object(3)
memory usage: 2.8+ MB
None


92308

In [34]:
pilpres[pilpres['username']=='jo_anezt'].count()

date        25
id          25
tweet       25
username    25
dtype: int64

In [23]:
print(pilpres.describe(include='all'))

                             date            id  \
count                       92308  9.230500e+04   
unique                      88907           NaN   
top     2022-04-21 08:00:02+00:00           NaN   
freq                          228           NaN   
mean                          NaN  1.552239e+18   
std                           NaN  3.309102e+16   
min                           NaN  1.477076e+18   
25%                           NaN  1.527496e+18   
50%                           NaN  1.559446e+18   
75%                           NaN  1.580639e+18   
max                           NaN  1.597741e+18   

                                                    tweet username  
count                                               92305    92302  
unique                                              90675    22391  
top     Anggota DPR Fraksi PDIP Johan Budi menilai sos...  jpnncom  
freq                                                   30     1069  
mean                                      

In [36]:
pilpres.isna().sum()

date        0
id          3
tweet       3
username    6
dtype: int64

In [37]:
print(pilpres.username.unique())

['Aprilia_wiji' 'GRANDFATHER_71' 'jo_anezt' ... 'wahyu_minarno'
 'Azhari25_' 'ninovendri']


In [38]:
print(pilpres.tweet.duplicated(keep='first').sum())

1632


## CLEANING DATA
untuk melakukan sentimen analisis yang benar maka harus melakukan pembersihan data agar bisa menghindari bias

In [40]:
#karena yang null hanya ada 6 baris dari 92308 data, maka sebaiknya drop row karena hanya sekian persen dari data dan tidak memiliki pengaruh besar pada hasil sentiment nanti
pil = pilpres.dropna()
len(pil)

92302

In [41]:
#pengecekan apakah masih ada baris yang null
pil.isnull().sum()
print(pil.isnull().sum())
print(pil.head())

date        0
id          0
tweet       0
username    0
dtype: int64
                        date            id  \
0  2022-11-29 23:54:52+00:00  1.597741e+18   
1  2022-11-29 23:54:24+00:00  1.597741e+18   
2  2022-11-29 23:46:58+00:00  1.597739e+18   
3  2022-11-29 23:44:12+00:00  1.597738e+18   
4  2022-11-29 23:39:45+00:00  1.597737e+18   

                                               tweet        username  
0  Menurut Andi Septa Wijaya, yang merupakan Deka...    Aprilia_wiji  
1  Presiden : "Biarin aja. Kita tambah penderitaa...  GRANDFATHER_71  
2  Rekam jejak Menteri BUMN Erick Thohir sebagai ...        jo_anezt  
3  @cagubnyinyir2 @aniesbaswedan @suryasena53 @ka...         AtukKe3  
4  Dr. KH. Haedar Nashir :\n\nPak Jokowi, negara ...       Utero2022  


# MENGHAPUS TWEET MASSAL YANG SAMA #
agar tidak mendapatkan bias maka harus menghapus tweet yang sama yang dilakukan oleh multi account

In [42]:
pil1 = pil.drop_duplicates(subset='tweet')
print(pil1.head())
len(pil1)

                        date            id  \
0  2022-11-29 23:54:52+00:00  1.597741e+18   
1  2022-11-29 23:54:24+00:00  1.597741e+18   
2  2022-11-29 23:46:58+00:00  1.597739e+18   
3  2022-11-29 23:44:12+00:00  1.597738e+18   
4  2022-11-29 23:39:45+00:00  1.597737e+18   

                                               tweet        username  
0  Menurut Andi Septa Wijaya, yang merupakan Deka...    Aprilia_wiji  
1  Presiden : "Biarin aja. Kita tambah penderitaa...  GRANDFATHER_71  
2  Rekam jejak Menteri BUMN Erick Thohir sebagai ...        jo_anezt  
3  @cagubnyinyir2 @aniesbaswedan @suryasena53 @ka...         AtukKe3  
4  Dr. KH. Haedar Nashir :\n\nPak Jokowi, negara ...       Utero2022  


90674

In [99]:
type(pil1)
print(pil1.tweet.duplicated().sum())

0


DATA TWEET Duplicate sudah dihapus